## Success Rates

Success Rates:  Using success rates to measure efficiency is superior to traditional yards per play (per pass or per rush) because it adds a layer of context to the play:  the down and distance to go.  Example:  a 10 yard run on 3rd and 26 will add 10 yards/carry to a running back's average.  But that play resulted in 4th and 16 and thus, an unsuccessful attempt to convert a first down (and graded as an unsuccessful play).  

Successful Plays = Frequency that a play generates required ydg (based on down/distance) to grade as successful

Source: https://www.sharpfootballstats.com/basics-and-faq.html

Per [Football Outsiders](http://www.footballoutsiders.com/stat-analysis/2004/introducing-running-back-success-rate), a run is considered a success if it satisfies the following benchmarks:

1. Gains 40% of yards needed on first down, 60% of yards needed on second down, or 100% of yards needed on third down

2. If the offense is behind by more than a touchdown in the fourth quarter, the first/second/third down benchmarks change to 50%/65%/100%, respectively.

3. If the offense is ahead by any amount in the fourth quarter, the first/second/third down benchmarks change to 30%/50%/100%, respectively.


![](https://www.4for4.com/fantasy-football/sites/default/files/u26881/ezgif-3-9ac278dbac.gif)

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 200) # Display up to 120 columns of a dataframe

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline
from matplotlib import style

Import data from the great [ryurko / nflscrapR-data](https://github.com/ryurko/nflscrapR-data). Your the best!

In [26]:
nfl = pd.read_csv('reg_pbp_2018.csv')
nfl.head()

play_id     game_id home_team away_team posteam posteam_type defteam  \
0       37  2018090600       PHI       ATL     ATL         away     PHI   
1       52  2018090600       PHI       ATL     ATL         away     PHI   
2       75  2018090600       PHI       ATL     ATL         away     PHI   
3      104  2018090600       PHI       ATL     ATL         away     PHI   
4      125  2018090600       PHI       ATL     ATL         away     PHI   

  side_of_field  yardline_100   game_date  quarter_seconds_remaining  \
0           PHI          35.0  2018-09-06                        900   
1           ATL          75.0  2018-09-06                        900   
2           ATL          80.0  2018-09-06                        900   
3           ATL          70.0  2018-09-06                        862   
4           ATL          59.0  2018-09-06                        826   

   half_seconds_remaining  game_seconds_remaining game_half  quarter_end  \
0                    1800                    3600     Half1            0   
1                    1800                    3600     Half1            0   
2                    1800                    3600     Half1            0   
3                    1762                    3562     Half1            0   
4                    1726                    3526     Half1            0   

   drive  sp  qtr  down  goal_to_go      time   yrdln  ydstogo  ydsnet  \
0      1   0    1   NaN         0.0  15:00:00  PHI 35        0      73   
1      1   0    1   1.0         0.0  15:00:00  ATL 25       10      73   
2      1   0    1   1.0         0.0  15:00:00  ATL 20       15      73   
3      1   0    1   2.0         0.0  14:22:00  ATL 30        5      73   
4      1   0    1   1.0         0.0  13:46:00  ATL 41       10      73   

                                                desc play_type  yards_gained  \
0  J.Elliott kicks 65 yards from PHI 35 to end zo...   kickoff             0   
1  (15:00) PENALTY on ATL-L.Paulsen, False Start,...   no_play             0   
2  (15:00) M.Ryan pass short right to J.Jones pus...      pass            10   
3  (14:22) J.Jones left end pushed ob at ATL 41 f...       run            11   
4  (13:46) D.Freeman right end to PHI 39 for 20 y...       run            20   

   shotgun  no_huddle  qb_dropback  qb_kneel  qb_spike  qb_scramble  \
0        0          0          0.0         0         0            0   
1        0          0          0.0         0         0            0   
2        0          0          1.0         0         0            0   
3        0          0          0.0         0         0            0   
4        0          0          0.0         0         0            0   

  pass_length pass_location  air_yards  yards_after_catch run_location  \
0         NaN           NaN        NaN                NaN          NaN   
1         NaN           NaN        NaN                NaN          NaN   
2       short         right        8.0                2.0          NaN   
3         NaN           NaN        NaN                NaN         left   
4         NaN           NaN        NaN                NaN        right   

  run_gap field_goal_result  kick_distance extra_point_result  \
0     NaN               NaN            NaN                NaN   
1     NaN               NaN            NaN                NaN   
2     NaN               NaN            NaN                NaN   
3     end               NaN            NaN                NaN   
4     end               NaN            NaN                NaN   

  two_point_conv_result  home_timeouts_remaining  away_timeouts_remaining  \
0                   NaN                        3                        3   
1                   NaN                        3                        3   
2                   NaN                        3                        3   
3                   NaN                        3                        3   
4                   NaN                        3                        3   

   timeout

## Team Stats

In [29]:
# indicate offensive plays
offensive_plays = ['pass', 'run']
nfl.loc[nfl.play_type.isin(offensive_plays), 'off_plays'] = 1
nfl.loc[nfl.desc.str.contains(' sacked'), 'off_plays'] = 1
nfl.loc[nfl.desc.str.contains(' sacked'), 'sack'] = 1
nfl['off_play_yds'] = np.where(nfl.off_plays== 1, nfl.yards_gained, 0)

nfl['first_down'] = nfl['first_down_rush'] + nfl['first_down_pass'] + nfl['first_down_penalty']
nfl.loc[(nfl.pass_touchdown == 1) | (nfl.rush_touchdown == 1) | (nfl.field_goal_result == 'made'), 'off_score_drive'] = 1

# indicate play attempt
nfl.loc[(nfl.pass_attempt == 1) | (nfl.rush_attempt == 1), 'play_attempt'] = 1

In [30]:
team_ = nfl.groupby(['posteam']).agg({
    'posteam_score': 'last',
    'yards_gained':sum,
    'off_plays':sum,
    'rush_attempt': sum,
    'pass_attempt':sum,
    'sack':sum,
    'off_play_yds': sum,
    'fumble_lost':sum,
    'first_down_rush': sum,
    'first_down':sum,
    'drive': lambda x: x.nunique(),
    'play_id': 'count',
    'off_score_drive':sum,
    'posteam_score':'last'
    
    
    
})

# may need to include: (1) Turnover = ['interception']


# Yards Per Offensive Play
#(Rush + Pass Yards) / (Pass Attempts + Rush Attempts + Times Sacked)
team_['yard_per_play'] = (team_.off_play_yds) / (team_.pass_attempt + team_.rush_attempt + team_.sack)



################
#Drive Performance

# % of offensive drives ending in a touchdown [pfr]
"""
[TODO: Recalulate] Note sure if correctly calcuating this measure
"""
team_['drive_per_score'] = team_.drive / team_.off_score_drive


team_['play_per_drive'] = team_.play_id / team_.drive
team_['yards_per_drive'] = team_.yards_gained / team_.drive
team_['pts_per_drive'] = team_.posteam_score / team_.drive


# Ranks
team_['yds_rk'] = team_.groupby('posteam')['yards_gained'].sum().rank(ascending=True)


team_

posteam_score  yards_gained  off_plays  rush_attempt  pass_attempt  \
posteam                                                                       
ATL               12.0           299       66.0          18.0          47.0   
BAL               47.0           369       75.0          34.0          41.0   
BUF                3.0           153       61.0          22.0          39.0   
HOU               13.0           293       64.0          32.0          32.0   
JAX               19.0           305       59.0          28.0          34.0   
NE                27.0           369       67.0          27.0          40.0   
NO                38.0           479       61.0          14.0          47.0   
NYG               15.0           324       63.0          24.0          39.0   
PHI               16.0           234       64.0          28.0          37.0   
TB                47.0           529       57.0          34.0          28.0   

         sack  off_play_yds  fumble_lost  first_down_rush  first_down  drive  \
posteam                                                                        
ATL       6.0           299          1.0              4.0        16.0     12   
BAL       2.0           369          1.0             11.0        26.0     14   
BUF       6.0           153          0.0              3.0        10.0     15   
HOU       3.0           294          2.0              9.0        17.0     14   
JAX       1.0           307          1.0              4.0        17.0     14   
NE        2.0           369          1.0              5.0        22.0     12   
NO        1.0           479          2.0              5.0        26.0     11   
NYG       2.0           324          0.0              3.0        15.0     12   
PHI       2.0           235          0.0              7.0        18.0     11   
TB        0.0           532          0.0              5.0        26.0     11   

         play_id  off_score_drive  yard_per_play  drive_per_score  \
posteam                                                             
ATL           88              3.0       4.211268         4.000000   
BAL          101              8.0       4.792208         1.750000   
BUF           90              1.0       2.283582        15.000000   
HOU           87              3.0       4.388060         4.666667   
JAX           86              3.0       4.873016         4.666667   
NE            87              5.0       5.347826         2.400000   
NO            86              6.0       7.725806         1.833333   
NYG           88              4.0       4.984615         3.000000   
PHI           91              3.0       3.507463         3.666667   
TB            86              7.0       8.580645         1.571429   

         play_per_drive  yards_per_drive  pts_per_drive  yds_rk  
posteam                                                          
ATL            7.333333        24.916667       1.000000     4.0  
BAL            7.214286        26.357143       3.357143     7.5  
BUF            6.000000        10.200000       0.200000     1.0  
HOU            6.214286        20.928571       0.928571     3.0  
JAX            6.142857        21.785714       1.357143     5.0  
NE             7.250000        30.750000       2.250000     7.5  
NO             7.818182        43.545455       3.454545     9.0  
NYG            7.333333        27.000000       1.250000     6.0  
PHI            8.272727        21.272727       1.454545     2.0  
TB             7.818182        48.090909       4.272727    10.0

In [31]:
team_.groupby('posteam')['yards_gained'].sum().rank(ascending=False)

posteam
ATL     7.0
BAL     3.5
BUF    10.0
HOU     8.0
JAX     6.0
NE      3.5
NO      2.0
NYG     5.0
PHI     9.0
TB      1.0
Name: yards_gained, dtype: float64

## Success Rate Calc

In [32]:
def success_rate_cal(play):    
    """
    [TODO: Further Analsyis Needed] Unsure what to do on Two Point Conversions
    """
    if play.ydstogo == 0:
        return 0
    
    yards_obtained = play.yards_gained / play.ydstogo

    if play.first == 1:
        return 1
    
    # check if play is if the fourth quarter
    elif play.qtr == 4:
        
        # check if offense is down by more than a TD
        if play.score_differential >= -7:
            if play.down == 1:
                if yards_obtained >= .50:
                    return 1
                else: return 0

            elif play.down == 2:
                if yards_obtained >= .65:
                    return 1
                else: return 0

            elif play.down >= 3:
                if yards_obtained >= 1:
                    return 1
                else: return 0
                
                
        # check if offense is down by more than a TD
        elif play.score_differential > 0:
            if play.down == 1:
                if yards_obtained >= .30:
                    return 1
                else: return 0

            elif play.down == 2:
                if yards_obtained >= .50:
                    return 1
                else: return 0

            elif play.down >= 3:
                if yards_obtained >= 1:
                    return 1
                else: return 0
        else:
            if play.down == 1:
                if yards_obtained >= .40:
                    return 1
                else: return 0

            elif play.down == 2:
                if yards_obtained >= .60:
                    return 1
                else: return 0

            elif play.down >= 3:
                if yards_obtained >= 1:
                    return 1
                else: return 0
            
    else:
        if play.down == 1:
            if yards_obtained >= .40:
                return 1
            else: return 0

        elif play.down == 2:
            if yards_obtained >= .60:
                return 1
            else: return 0

        elif play.down >= 3:
            if yards_obtained >= 1:
                return 1
            else: return 0

In [33]:
nfl['success_play'] = nfl[nfl.play_attempt==1].apply(success_rate_cal, axis=1)
nfl.head()

play_id     game_id home_team away_team posteam posteam_type defteam  \
0       37  2018090600       PHI       ATL     ATL         away     PHI   
1       52  2018090600       PHI       ATL     ATL         away     PHI   
2       75  2018090600       PHI       ATL     ATL         away     PHI   
3      104  2018090600       PHI       ATL     ATL         away     PHI   
4      125  2018090600       PHI       ATL     ATL         away     PHI   

  side_of_field  yardline_100   game_date  quarter_seconds_remaining  \
0           PHI          35.0  2018-09-06                        900   
1           ATL          75.0  2018-09-06                        900   
2           ATL          80.0  2018-09-06                        900   
3           ATL          70.0  2018-09-06                        862   
4           ATL          59.0  2018-09-06                        826   

   half_seconds_remaining  game_seconds_remaining game_half  quarter_end  \
0                    1800                    3600     Half1            0   
1                    1800                    3600     Half1            0   
2                    1800                    3600     Half1            0   
3                    1762                    3562     Half1            0   
4                    1726                    3526     Half1            0   

   drive  sp  qtr  down  goal_to_go      time   yrdln  ydstogo  ydsnet  \
0      1   0    1   NaN         0.0  15:00:00  PHI 35        0      73   
1      1   0    1   1.0         0.0  15:00:00  ATL 25       10      73   
2      1   0    1   1.0         0.0  15:00:00  ATL 20       15      73   
3      1   0    1   2.0         0.0  14:22:00  ATL 30        5      73   
4      1   0    1   1.0         0.0  13:46:00  ATL 41       10      73   

                                                desc play_type  yards_gained  \
0  J.Elliott kicks 65 yards from PHI 35 to end zo...   kickoff             0   
1  (15:00) PENALTY on ATL-L.Paulsen, False Start,...   no_play             0   
2  (15:00) M.Ryan pass short right to J.Jones pus...      pass            10   
3  (14:22) J.Jones left end pushed ob at ATL 41 f...       run            11   
4  (13:46) D.Freeman right end to PHI 39 for 20 y...       run            20   

   shotgun  no_huddle  qb_dropback  qb_kneel  qb_spike  qb_scramble  \
0        0          0          0.0         0         0            0   
1        0          0          0.0         0         0            0   
2        0          0          1.0         0         0            0   
3        0          0          0.0         0         0            0   
4        0          0          0.0         0         0            0   

  pass_length pass_location  air_yards  yards_after_catch run_location  \
0         NaN           NaN        NaN                NaN          NaN   
1         NaN           NaN        NaN                NaN          NaN   
2       short         right        8.0                2.0          NaN   
3         NaN           NaN        NaN                NaN         left   
4         NaN           NaN        NaN                NaN        right   

  run_gap field_goal_result  kick_distance extra_point_result  \
0     NaN               NaN            NaN                NaN   
1     NaN               NaN            NaN                NaN   
2     NaN               NaN            NaN                NaN   
3     end               NaN            NaN                NaN   
4     end               NaN            NaN                NaN   

  two_point_conv_result  home_timeouts_remaining  away_timeouts_remaining  \
0                   NaN                        3                        3   
1                   NaN                        3                        3   
2                   NaN                        3                        3   
3                   NaN                        3                        3   
4                   NaN                        3                        3   

   timeout

## NEED TO DO SOME GRAPHS ON THIS DISTRIBUTION!

### Set indicator for Week #s

In [5]:
import datetime
def game_weeks(df):
    """
    Create dataset indicate the game week. 
    """
    
    week_dict = {}
    first_game_date = df.min().date()
    game_date = first_game_date
    season_start_date = pd.to_datetime('2018-09-04 00:00:00').date()
    last_game_date = df.max().date()
    week_num = 1 # start week 1 and increment from there
    
    for i in range((last_game_date - first_game_date).days+1):
        if game_date <= season_start_date:
            week_dict[game_date] = 0
            game_date += datetime.timedelta(days=1)
        else:
            if game_date.weekday() == 1: # check if weekday is Tuesday.. if so reset week_number
                week_num += 1
            week_dict[game_date] = week_num
            game_date += datetime.timedelta(days=1)
    
    week_list = pd.Series(week_dict).to_frame()[0]
    
    return week_list

d = game_weeks(pd.to_datetime(nfl.game_date))
for o,j in enumerate(d.index):
    nfl.loc[nfl.game_date == str(j), 'week'] = d[o]

### Standings

In [20]:
win_loss_df = nfl.groupby(['game_id','home_team', 'away_team']).agg({'total_home_score':'last',
                                                        'total_away_score':'last',
                                                          }).reset_index()

#wins
win_loss_df.loc[win_loss_df.total_home_score > win_loss_df.total_away_score, 'home_win'] = 1
win_loss_df.loc[win_loss_df.total_home_score > win_loss_df.total_away_score, 'away_win'] = 0

#loses
win_loss_df.loc[win_loss_df.total_home_score < win_loss_df.total_away_score, 'home_loss'] = 0
win_loss_df.loc[win_loss_df.total_home_score < win_loss_df.total_away_score, 'away_loss'] = 1

#ties
win_loss_df.loc[win_loss_df.total_home_score == win_loss_df.total_away_score, 'home_tie'] = 1
win_loss_df.loc[win_loss_df.total_home_score == win_loss_df.total_away_score, 'away_tie'] = 1

win_loss_df.head()

game_id home_team away_team  total_home_score  total_away_score  \
0  2018090600       PHI       ATL                18                12   
1  2018090900       BAL       BUF                47                 3   
2  2018090905        NE       HOU                27                13   
3  2018090906        NO        TB                40                47   
4  2018090907       NYG       JAX                15                19   

   home_win  away_win  home_loss  away_loss  home_tie  away_tie  
0       1.0       0.0        NaN        NaN       NaN       NaN  
1       1.0       0.0        NaN        NaN       NaN       NaN  
2       1.0       0.0        NaN        NaN       NaN       NaN  
3       NaN       NaN        0.0        1.0       NaN       NaN  
4       NaN       NaN        0.0        1.0       NaN       NaN

In [14]:
win_loss_df.total_home_score[3] > win_loss_df.total_away_score[3]

False

In [21]:
nfl = nfl.merge(win_loss_df, on=['game_id','home_team','away_team'], how='left')

# Create a Team_Win column to indicate a win loss
nfl['team_win'] = nfl.apply(lambda x: x.home_win if x.home_team == x.posteam else x.away_win, axis=1)
nfl['team_loss'] = nfl.apply(lambda x: x.home_win if x.home_team == x.posteam else x.away_win, axis=1)
nfl['win_pct'] = nfl.team_win / (nfl.team_win + nfl.team_loss)

nfl = nfl.groupby(['posteam']).agg({'team_win':'count', 'team_loss':sum,'win_pct':'last',
                                 'wp':'mean'})


nfl.head()


team_win  team_loss  win_pct        wp
posteam                                        
ATL            88        0.0      NaN  0.470389
BAL           101      101.0      0.5  0.899482
BUF            90        0.0      NaN  0.051514
HOU            87        0.0      NaN  0.187235
JAX             0        0.0      NaN  0.665691

nfl.head()